In [8]:
import sys
sys.path
sys.path.append('../')

import numpy as np 
import pandas as pd

from data import RandomData, AmazonBooks, ToyData, MovieLensData
from model import SimpleMeanModel, UserMeanModel, ProductMeanModel, CombinedMeanModel

from nlu_feature_extractor import *
import re

%matplotlib inline

### Load Data

In [3]:
# ds = MovieLensData(min_user_ratings=5).get_dataset(verbose=True)
ds = AmazonBooks(min_user_ratings=5).get_dataset(verbose=True)
train = ds['train']
val = ds['val']

loading preprocessed dataset from disk


In [40]:
print(train.keys())
train['product_reviews'].head()

dict_keys(['user_product_ratings', 'product_descriptions', 'product_reviews'])


,product_id,review
0,6155,for getting your kid introduced to his/her ABC...
1,6155,"This Book is funny and is full of B words, lik..."
2,6155,A favorite Berenstain book of my children I wa...
3,6155,This book is quite funny. Especially when you...
4,6155,Teaching the next generation to love books! M...


## create matrix: product_id -> all text about that product

all text includes all reviews about it and the description of that product

In [4]:
# grouped_reviews = train['product_reviews'].groupby(['product_id']).get_group(6155)
# grouped_reviews['review'].agg([np.concatenate])
grouped_reviews = train['product_reviews'].groupby('product_id')['product_id', 'review'].aggregate(\
        {'product_id':['mean'], 'review':lambda x: list(x)})
grouped_reviews .columns = ['product_id', 'review']

In [5]:
grouped_reviews.head()

,product_id,review
product_id,,
2,2,[My three year old loves Dora the Explorer and...
7,7,[This is an excellent introduction to one of t...
8,8,"[The introduction by A. S. Byatt is fantastic,..."
14,14,[This revised and enlarged edition is an unusu...
15,15,[I think it's important for readers to separat...


In [250]:
# print(list(grouped_reviews.loc[grouped_reviews['product_id'] == 6155]['review'])[0])

In [6]:
combined = train['product_descriptions'].merge(grouped_reviews, on='product_id')
combined['all_text'] = combined['review']
combined['all_text'] = combined['description'].apply(lambda x: [x]) +  combined['all_text']
combined.head()

,description,product_id,review,all_text
0,The only complete on-the-scene account of the ...,3821,[I was given this book to read as an assignmen...,[The only complete on-the-scene account of the...
1,"""Fairy phobic or trapped in a true-life fairy ...",60922,[Dream come true or worst nightmare? Grace Mac...,"[""Fairy phobic or trapped in a true-life fairy..."
2,"In this lush, lyrical, and marvelously evocati...",142370,"[this novel is not your typical love story, wh...","[In this lush, lyrical, and marvelously evocat..."
3,Robert Englander is Principal Engineer and Pre...,121224,[1. Good coverage of SOAP2. Uses GLUE (acquire...,[Robert Englander is Principal Engineer and Pr...
4,"Text: English, Hebrew\tKerry M. Olitzky, D.H.L...",62060,[First rate scholarship and clarify of transla...,"[Text: English, Hebrew\tKerry M. Olitzky, D.H...."


In [9]:
combined['all_text_parsed'] = combined['all_text'].apply(lambda x: re.sub("[,.;?!():\[\]\"\"]",""," ".join(x)).lower())
combined['all_text_parsed_words_separate'] = combined['all_text_parsed'].apply(lambda x: x.split(" "))
print(combined.head())
# first join all comments reviews, etcetera into 1 big string
# then remove all punctuation
# then split on spaces to create words

                                         description  product_id  \
0  The only complete on-the-scene account of the ...        3821   
1  "Fairy phobic or trapped in a true-life fairy ...       60922   
2  In this lush, lyrical, and marvelously evocati...      142370   
3  Robert Englander is Principal Engineer and Pre...      121224   
4  Text: English, Hebrew\tKerry M. Olitzky, D.H.L...       62060   

                                              review  \
0  [I was given this book to read as an assignmen...   
1  [Dream come true or worst nightmare? Grace Mac...   
2  [this novel is not your typical love story, wh...   
3  [1. Good coverage of SOAP2. Uses GLUE (acquire...   
4  [First rate scholarship and clarify of transla...   

                                            all_text  \
0  [The only complete on-the-scene account of the...   
1  ["Fairy phobic or trapped in a true-life fairy...   
2  [In this lush, lyrical, and marvelously evocat...   
3  [Robert Englander is Princi

## extract  vocab

In [10]:
vocab = get_vocab(combined['all_text_parsed_words_separate'].tolist(), 5000)
print(len(vocab))
print(vocab)

5001
['', '\t', '\t\t\t\t\t\t\thardcover\t', '\t\t\t\t\t\t\tpaperback\t', '#1', '$UNK', '&', '&amp', '&quot', '&quota', '&quothow', '&quoti', '&quotthe', "'a", "'the", '+', '-', '--', '---', '--this', '/', '1', '1/2', '10', '100', '100%', '1000', '101', '11', '12', '13', '14', '15', '150', '16', '17', '18', "1800's", '18th', '19', '1995', '1996', '1997', '1998', '1999', '19th', '1st', '2', '20', '200', '2000', '2001', '2002', '2003', '2004', '2005', '20th', '21', '21st', '22', '23', '24', '25', '2nd', '3', '30', '300', '35', '3rd', '4', '40', '400', '45', '4th', '5', '50', '500', '5th', '6', '60', '6th', '7', '70', '7th', '8', '8th', '9', '9/11', '=', 'a', 'abilities', 'ability', 'able', 'about', 'above', 'abridged', 'absolute', 'absolutely', 'absorb', 'absorbed', 'absorbing', 'abstract', 'absurd', 'abuse', 'academic', 'academy', 'accept', 'acceptable', 'accepted', 'access', 'accessible', 'accident', 'acclaimed', 'accompanying', 'accomplish', 'accomplished', 'according', 'account', 'ac

## create document, vocab matrix

In [11]:
combined_words = combined[['product_id','all_text_parsed']]
for v in vocab:
    combined_words[v] = combined_words['all_text_parsed'].apply(lambda x: x.count(" " + v + " "))
combined_words

/home/shared/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


KeyboardInterrupt: 

In [ ]:
np.max(np.max(combined_words[vocab]))

### save vocab matrix

In [511]:
combined_words.to_pickle("vocab_matrix_2")  # where to save it, usually as a .pkl
# Then you can load it back using: df = pd.read_pickle(file_name)

## convert to tfidf

In [513]:
from sklearn.feature_extraction.text import TfidfTransformer
tf = TfidfTransformer()
tfidf_words = tf.fit_transform(combined_words[vocab])
tfidf_words

<6568x5001 sparse matrix of type '<class 'numpy.float64'>'
	with 1745315 stored elements in Compressed Sparse Row format>

In [514]:
tfidf_df = pd.SparseDataFrame(tfidf_words)
tfidf_df.columns = vocab
tfidf_df["product_id"] = combined_words["product_id"]
tfidf_df = tfidf_df[["product_id"] + vocab]
tfidf_df.head()

,product_id,,,paperback,#1,#2,$UNK,&,&#34the,&amp,...,young,younger,youngest,your,yours,yourself,youth,yr,zen,zero
0,12572,0.249810,NaN,NaN,0.036692,NaN,NaN,NaN,NaN,NaN,...,0.020471,NaN,NaN,0.023859,NaN,NaN,NaN,NaN,NaN,NaN
1,4397,0.271908,NaN,0.028557,0.035620,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.038753
2,4175,0.083713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.045276,NaN,NaN,0.052770,NaN,NaN,0.079512,NaN,NaN,NaN
3,9448,0.051491,NaN,NaN,NaN,NaN,NaN,0.026269,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14535,0.177852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### save tfidf

In [679]:
tfidf_df.to_pickle("tfidf_matrix")  # where to save it, usually as a .pkl

In [683]:
pickle.dump( tf.get_params(), open( "tf_params.p", "wb" ) )